# Directed Search: Small-Difference Equilibrium with Y1 > Y2

James Yu, 9 February 2021

This brief notebook expands on the mechanism from [this previous notebook](https://github.com/jbrightuniverse/Directed-Search-with-Piecewise-Pi/blob/main/DirectedSearchPiecewiseNumerical.ipynb) to resolve an issue with missing components of the piecewise $\pi$. It also formulates a solution for problem 3 of [this assignment](https://montoya.econ.ubc.ca/Econ514/problem_set_2.pdf).

[This other notebook](https://github.com/michaelpetersubc/notebooks/blob/master/Econ306/directed_search/directed_search_incomplete_information.ipynb) by Michael Peters describes the full extent of the solution for the Nash equilibrium $\pi_h$ and $\pi_l$ which is implemented here.

Directly using Python was found to be too slow to compute within a timely manner, and attempting to apply a JIT compiler failed due to an inability to type-detect so to speed the computation up, it was written in C++ and then executed via Python in this notebook.

First, we define our $\pi_h$ and $\pi_l$, which are piecewise based on inputs $w_1$, $w_2$ and $\lambda$. Here we use gamma instead of $\lambda$ to avoid issues with lambda being a reserved method name.

In [1]:
myfile = """
    #include <math.h>
    #include <fstream>

    float pi_h(float w1, float w2, float gamma) {
      if (gamma*w1/2 + (1-gamma)*w1 >= w2)
        return 1;
      else if (w1 > w2 && gamma*w1/2 + (1-gamma)*w1 <= w2)
        return ((gamma-2)*w2 + 2*w1)/(gamma*(w1+w2));
      else if (w2 > w1 && gamma*w2/2 + (1-gamma)*w2 <= w1)
        return ((gamma-2)*w1 + 2*w2)/(gamma*(w2+w1));
      return 0;
    }

    float pi_l(float w1, float w2, float gamma) {
      if (gamma*w1/2 + (1-gamma)*w1 >= w2)
        return ((1-gamma)*(2*w1 - w2)) / ((w1+w2)-gamma*(w1-w2));
      else if (w1 > w2 && gamma*w1/2 + (1-gamma)*w1 <= w2)
        return ((3-gamma)*w2 - 2*w1)/((1-gamma)*(w1+w2));
      else if (w2 > w1 && gamma*w2/2 + (1-gamma)*w2 <= w1)
        return ((3-gamma)*w1 - 2*w2)/((1-gamma)*(w2+w1));
      return ((1-gamma)*(2*w2 - w1)) / ((w2+w1)-gamma*(w2-w1));
    }
"""

We include the math header for access to `pow()` to do exponentiation, and we include fstream to write to files.

Next, we implement our profit functions. The profit function for each firm is dependent on the probability of at least one individual applying to each firm and is computed using relevant combinations of $\lambda$ and $\pi$.

In [2]:
myfile += """
    float profit_1(float w1, float w2, float gamma, float y1, float hp, float lp) {
      float blob = pow(gamma, 2) * (1-pow(hp, 2));
      blob += pow((1-gamma), 2) * (1-pow(lp, 2));
      blob += 2*gamma*(1-gamma)*(1-hp)*(1-lp);
      return (y1-w1)*(1-blob);
    }

    float profit_2(float w1, float w2, float gamma, float y2, float hp, float lp) {
      float blob = pow(gamma, 2) * pow(hp, 2);
      blob += pow((1-gamma), 2) * pow(lp, 2);
      blob += 2*gamma*(1-gamma)*(hp)*(lp);
      return (y2-w2)*(1-blob);
    }
"""

Next, we run the algorithm to find the profit-maximizing choices of $w_1$ and $w_2$. This is done by a brute-force grid search over ranges of $w_1$, $w_2$ and $\lambda$ from 0 to 1, plus revenue $y$ from 0 to 10. The results of computing the profit functions for the two firms over our grid are stored in two four-dimensional arrays.

In [3]:
myfile += """
    int main(void) {
      auto profit1 = new float[100][100][100][20];
      auto profit2 = new float[100][100][100][20];
      int i = 0, j, k, l;
      for (float w1 = 0.0000001; i < 100; w1 += 0.01) {
        int j = 0;
        for (float w2 = 0.0000001; j < 100; w2 += 0.01) {
          int k = 0;
          for (float gamma = 0.0000001; k < 100; gamma += 0.01) {
            float hp = pi_h(w1, w2, gamma);
            float lp = pi_l(w1, w2, gamma);
            int l = 0;
            for (float y = 0; l < 20; y += 0.5) {
              profit1[i][j][k][l] = profit_1(w1, w2, gamma, y, hp, lp);
              profit2[i][j][k][l] = profit_2(w1, w2, gamma, y, hp, lp);
              l++;
            }
            k++;
          }
          j++;
        }
        i++;
      }
"""

Next, we compute the best-response wage choices for each firm. A best response is a wage choice which maximizes profit given the other firm chose a particular wage of their own. We iterate over all entries of the computed grid and compare each wage vector for every $w$-$\lambda$-$y$ pair where $w$, represented here as the $i$ index, is the wage choice of the other firm; and $y$, represented here as the $k$ index, is the revenue that our firm makes.

The wage which gives the highest profit in a particular row is saved as the best response.

In [4]:
myfile += """
      auto bestresponse1 = new float[100][100][20];
      auto bestresponse2 = new float[100][100][20];

      for (int i = 0; i < 100; i++) {
        for (int j = 0; j < 100; j++) {
          for (int k = 0; k < 20; k++) {
            int argmax_1 = 0, argmax_2 = 0;
            float mymax_1 = 0, mymax_2 = 0;
            for (int w = 0; w < 100; w++) {
              float entry = profit1[w][i][j][k];
              if (entry > mymax_1) {
                mymax_1 = entry;
                argmax_1 = w;
              }
              entry = profit2[i][w][j][k];
              if (entry > mymax_2) {
                mymax_2 = entry;
                argmax_2 = w;
              }
            }
            bestresponse1[i][j][k] = argmax_1;
            bestresponse2[i][j][k] = argmax_2;
          }
        }
      }
"""

Finally, we find the Nash equilibria. The Nash equilibria are points where firm 1 and firm 2 choose wages which are best-responses to each other. We iterate over the best-response grid, adjusted to account for firm 1 and firm 2 potentially having different revenue levels (hence $k1$ and $k2$ for $y1$ and $y2$, respectively), and if we find a Nash equilibrium, we save the index data to a file.

In [5]:
myfile += """

      std::ofstream myfile;
      myfile.open("data.txt");
      for (int i = 0; i < 100; i++) {
        for (int j = 0; j < 100; j++) {
          for (int k1 = 0; k1 < 20; k1++) {
            for (int k2 = 0; k2 < 20; k2++) {
              int arg = bestresponse1[i][j][k1];
              if (bestresponse2[arg][j][k2] == i) {
                myfile << arg << "," << i << "," << j << "," << k1 << "," << k2 << "\\n";
              }
            }
          }
        }
      }

      myfile.close();
      return 0;
    }
"""

Next, we save the C++ code and compile it using g++. We then run it, which will output some data to `data.txt`.

In [6]:
with open("program.cpp", "w") as f:
    f.write(myfile)

In [7]:
import os
os.system("g++ -o program program.cpp")
os.system("./program")

0

The 0 means the syscall worked properly. Thus, we can open the data file and read it into a list.

In [8]:
res = []
with open("data.txt") as f:
    for line in f:
        res.append([int(b) for b in line.replace("\n", "").split(",")])

In [9]:
print(len(res))

19715


Next, we reconstruct the mapping between grid indices and actual wage values, along with revenue values, to remap the indices from the data file to actual values. We then run the actual checks from problem 3 of the assignment to see if the argument held up. Before doing so, we run one filter which ensures $y_1$ > $y_2$.

In [10]:
ws = []
w = 0.0000001
for i in range(100):
    ws.append(w)
    w += 0.01
    
ys = [i/2 for i in range(20)]
ws

for i in range(len(res)):
    data = res[i]
    newdata = [ws[data[0]], ws[data[1]], ws[data[2]], ys[data[3]], ys[data[4]]]
    res[i] = newdata
    
res = list(filter(lambda x: x[3] > x[4], res))
print(len(res))

res = list(filter(lambda x: x[2]*x[0]/2+(1-x[2])*x[0] <= x[1], res))
print(len(res))

res = list(filter(lambda x: x[0] > x[1], res))
print(len(res))

17648
16517
15009


If the argument from problem 3 of the assignment held, we should not have seen a reduction in the size of the list. This means something went wrong.